In [1]:
import os
import math
import random
from PIL import Image, ImageChops
import pandas as pd

In [90]:
IMAGE_NAMES = os.listdir("photos")
IMAGE_NAMES.remove('.DS_Store')

# Load images from the "photos" dir
images = [Image.open(f"photos/{image}") for image in IMAGE_NAMES][:]
shapes = [image.size for image in images]

In [91]:
tot_images = len(images)
cols = 10
rows = math.ceil(tot_images/cols)
print(f"Tot:{tot_images} Rows:{rows} Cols: {cols}")

Tot:115 Rows:12 Cols: 10


In [92]:
if tot_images < rows*cols:
    # If we have missing photos to complete the image,
    # sample randomly few images
    missing_images =  (rows*cols) - tot_images

    for i in range(missing_images):
        img = random.randint(0, len(images))
        images.append(images[img])

In [93]:
thmb_dim = pd.DataFrame(shapes).min()

# Reshape images using thumbnails
img_resize = [image.thumbnail((thmb_dim[0], thmb_dim[1])) for image in images]

width = math.ceil(cols * thmb_dim[0])
height = math.ceil(rows * thmb_dim[1])

#Light blue
color = (109,172,249)
#Purple
#color = (98,37,153)

new_image = Image.new("RGB", size = (width, height), color=color)

i = 0 
# Leave space on the left and the right of the photo
for n in range(rows):
    for m in range(cols):
        position = (math.ceil(m*thmb_dim[0]), math.ceil(n*thmb_dim[1]))
        new_image.paste(images[i],position)
        i+=1

new_image.show()

# Save image
with open("./spaced_img.png", mode="wb") as f:
    new_image.save(f)

In [94]:
thmb_dim = pd.DataFrame(shapes).min()

# Scale images using thumbnails
img_resize = [image.thumbnail((thmb_dim[0], thmb_dim[1])) for image in images]
img_resize = []

# Removes the borders added by the thumbnail rescaling. This is needed to align all the
# images on the left
for image in images:
    diff = ImageChops.difference(image, Image.new("RGB", image.size, color=color))
    diff = ImageChops.add(image, diff, 2, -100)
    bbox = diff.getbbox()
    if bbox:
        img_resize.append(image.crop(bbox))

width = math.ceil(cols * thmb_dim[0])
height = math.ceil(rows * thmb_dim[1])

#Light blue
color = (109,172,249)
#Purple 
#color = (98,37,153)

new_image = Image.new("RGB", size = (width, height), color=color)

i = 0 
for n in range(rows):
    x = 0
    for m in range(cols):
        y = math.ceil(n*thmb_dim[1])
        position = (x, y)
        new_image.paste(images[i],position)
        # Collapse all the photos on the left
        x += img_resize[i].size[0]
        i+=1

new_image.show()

# Save image
with open("./collapsed_img.png", mode="wb") as f:
    new_image.save(f)

In [97]:
# Compress the final image and save it
new_image.thumbnail((2000, 2000))
with open("compressed_img.png", mode="wb") as f:
    new_image.save(f)